### Feature Engineering - Combined
This notebook combines all features from the various Feature Engineering notebooks.

The Result:
    - train-featurized.csv & test-featurized.csv - the original train.csv and test.csv combined with all available features

In [1]:
# PyData
import pandas
import numpy
from pandas import DataFrame, Series
from sklearn import datasets, linear_model, preprocessing, cross_validation
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, Ridge

# System
import datetime
import os
import math

# Graphing
#%matplotlib inline # Only works on Python 3 in the docker container
#import seaborn # Only works on Python 3 in the docker container

#os.environ['PYSPARK_PYTHON'] = 'python2'

%matplotlib inline


# Featurize Training Data

In [2]:
df_basic = DataFrame.from_csv("train_features_basic.csv", index_col=['Date', 'Store'])
df_means = DataFrame.from_csv("train-features-predicted_mean.csv", index_col=['Date', 'Store'])
df_customers = DataFrame.from_csv("train-features-predicted_mean-customers.csv", index_col=['Date', 'Store'])

In [3]:
df_sales = DataFrame.from_csv("train.csv", index_col=['Date', 'Store'])[['Sales', 'Customers']]

/opt/conda/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [4]:
df_features = df_means.join(df_basic, how='outer')
train_df = df_sales.join(df_features)

In [5]:
train_df['Sales_predicted'] = train_df['Sales_predicted'].fillna(train_df['Sales_predicted'].mean())
train_df['Sales_predicted'] = train_df['Sales_predicted'].map(lambda x: int(x))

train_df['Sales_predicted_by_mean'] = train_df['Sales_predicted']
del train_df['Sales_predicted']

In [6]:
train_df.head()

Sales  Customers  DayOfWeek  Open  Promo  StateHoliday  \
Date       Store                                                           
2015-07-31 1       5263        555          5     1      1             0   
           2       6064        625          5     1      1             0   
           3       8314        821          5     1      1             0   
           4      13995       1498          5     1      1             0   
           5       4822        559          5     1      1             0   

                  SchoolHoliday  cal:dayOfMonth  cal:dayOfWeek  cal:dayofyear  \
Date       Store                                                                
2015-07-31 1                  1              31              4            212   
           2                  1              31              4            212   
           3                  1              31              4            212   
           4                  1              31              4            212   
           5                  1              31              4            212   

                  cal:month  cal:quarter  cal:weekofyear  cal:year  \
Date       Store                                                     
2015-07-31 1              7            3              31      2015   
           2              7            3              31      2015   
           3              7            3              31      2015   
           4              7            3              31      2015   
           5              7            3              31      2015   

                  Sales_predicted_by_mean  
Date       Store                           
2015-07-31 1                         5145  
           2                         5115  
           3                         8138  
           4                        10275  
           5                         5308

In [7]:
train_df.to_csv("train-featurized.csv")

# Featurize Test Data

In [8]:
df_basic = DataFrame.from_csv("test_features_basic.csv", index_col=['Date', 'Store'])
df_means = DataFrame.from_csv("test-features-predicted_mean.csv", index_col=['Date', 'Store'])
df_customers = DataFrame.from_csv("test-features-predicted_customers.csv", index_col='Id')


In [9]:
del df_basic['Id']
test_df = DataFrame.from_csv("test.csv", index_col=['Date', 'Store'])[['Id']]

In [10]:
test_df = test_df.join(df_basic, how='left')

In [11]:
test_df = test_df.join(df_means, how='left')

In [12]:
test_df.reset_index(inplace=True)
test_df.set_index("Id", inplace=True)

In [13]:
test_df = test_df.join(df_customers, how='left')

In [14]:
test_df['Sales_predicted'] = test_df['Sales_predicted'].fillna(test_df['Sales_predicted'].mean())
test_df['Sales_predicted'] = test_df['Sales_predicted'].map(lambda x: int(x))

test_df['Sales_predicted_by_mean'] = test_df['Sales_predicted']
del test_df['Sales_predicted']

In [15]:
test_df['Customers'] = test_df['Prediction']
del test_df['Prediction']

In [16]:

test_df['Open'] = test_df['Open'].fillna(1)

In [17]:
test_df.head()

,Date,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,cal:dayOfMonth,cal:dayOfWeek,cal:dayofyear,cal:month,cal:quarter,cal:weekofyear,cal:year,Sales_predicted_by_mean,Customers
Id,,,,,,,,,,,,,,,,
1,2015-09-17,1,4,1,1,0,0,17,3,260,9,3,38,2015,4961,528.441265
2,2015-09-17,3,4,1,1,0,0,17,3,260,9,3,38,2015,7994,799.069934
3,2015-09-17,7,4,1,1,0,0,17,3,260,9,3,38,2015,9406,950.083264
4,2015-09-17,8,4,1,1,0,0,17,3,260,9,3,38,2015,7194,817.549724
5,2015-09-17,9,4,1,1,0,0,17,3,260,9,3,38,2015,6777,572.575898


In [18]:
test_df.to_csv("test-featurized.csv")